# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Event-Driven Architecture

**Subtopic** - Implementing a Simple Event-Driven System

**Use Case** - Implement a priority queue in Redis that assigns different priorities to events based on their type and urgency, ensuring high-priority events are processed first.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
redis==6.2.0
```


# Prompt

I am giving you a stream of real-time events with associated urgency and type. Your task is to implement a Redis-backed priority queue system in Python that processes these events in strict priority order.

The priorty order is as follows:
1. Urgency (higher = more important)
2. Event Type (custom ranked: 'alert' > 'transaction' > 'log')
3. FIFO (insertion order tie-breaker)

**Input Format**

Each event is a Python dictionary with the following fields:
- event_id: Unique identifier for the event.
- event_type: could be 'alert', 'transaction', or 'log'.
- urgency: A number from 1 to 5.
- payload: JSON-encoded string containing the event’s content.

**Output Format**

A sequence of event IDs in the exact order they are dequeued from the priority queue.

The output list must reflect the correct priority handling based on:
- Highest urgency first,
- Within the same urgency: event_type precedence 'alert' > 'transaction' > 'log',
- Within the same urgency and type: FIFO (earlier enqueued first).

**Examples**

```python
# Input Format
enqueue_event(redis_conn, {
    "event_id": "evt1",
    "event_type": "transaction",
    "urgency": 3,
    "payload": "{\"amount\": 250}"
})

enqueue_event(redis_conn, {
    "event_id": "evt2",
    "event_type": "alert",
    "urgency": 3,
    "payload": "{\"msg\": \"Disk space low\"}"
})

enqueue_event(redis_conn, {
    "event_id": "evt3",
    "event_type": "log",
    "urgency": 5,
    "payload": "{\"info\": \"User logged in\"}"
})

enqueue_event(redis_conn, {
    "event_id": "evt4",
    "event_type": "transaction",
    "urgency": 5,
    "payload": "{\"amount\": 1000}"
})

# Output Format
[
    "evt4",   # urgency 5, transaction
    "evt3",   # urgency 5, log
    "evt2",   # urgency 3, alert
    "evt1"    # urgency 3, transaction
]

```

# Requirements

**Explicit and Implicit Points**

- Events must be prioritized by urgency > event_type > insertion order.
- Priority must be encoded into a single numerical score suitable for Redis Sorted Sets (ZADD).
- Insertion order must be preserved for tie-breaking when urgency and type are identical.
- Event objects must be stored as JSON strings in Redis.
- Use Redis Sorted Sets (ZADD, ZPOPMIN) for queue operations.
- No additional libraries are allowed except redis-py.

**Solution Expectations**

- Make a priority scoring system combining all three factors.
- Use a Redis-compatible serialization format (JSON) for events.
- Use ZADD to insert events into a sorted set queue and ZPOPMIN to dequeue them.
- Preserve deterministic ordering for events with same urgency and type.a
- Events must be unique by event_id, no duplicates are allowed in the queue.

**Function Signatures**

```python
def enqueue_event(redis_conn, event: dict) -> None:
    pass

def dequeue_event(redis_conn) -> Optional[dict]:
    pass
```

**Edge Case Behavior**

- If the queue is empty, dequeue_event should return empty list.
- If multiple events share urgency and event_type, they must dequeue in the order they were enqueued.
- Events with unknown event_type values can be treated as lowest priority.
- if an unknown event enters the queue it should be handled with exception handling instead of crashing the program.
- Timestamp collisions or same-priority events must not break FIFO ordering.
- All stored events must be retrievable in original structure when dequeued.

**Constraints**

- 1 <= urgency <= 5.
- allowed event types are only 'alert', 'transaction', 'log'.
- Redis is always available and responsive.
- Only one priority queue is used for all event types.
- No third-party libraries except redis-py.
- Must use Redis Sorted Set data type only (no in-memory queues).
- Must handle thousands of enqueues without breaking FIFO behavior.

In [ ]:
# code

"""This program simulates a priority queue system using redis db."""

import json
import redis
from typing import Optional, Dict

EVENT_TYPE_RANK = {
    "alert": 0,
    "transaction": 1,
    "log": 2
}

PRIORITY_QUEUE_KEY = "priority_event_queue"
COUNTER_KEY = "priority_event_counter"


def enqueue_event(redis_conn: redis.Redis, event: Dict) -> None:
    """Enqueue an event into a Redis priority queue based on priority."""
    event_id = event.get("event_id")
    event_type = event.get("event_type")
    urgency = event.get("urgency")
    payload = event.get("payload")

    if not event_id:
        raise ValueError("Missing 'event_id' in event")
    if event_type not in EVENT_TYPE_RANK:
        raise ValueError(f"Unknown event_type '{event_type}'")
    if not isinstance(urgency, int) or not (1 <= urgency <= 5):
        raise ValueError("Urgency must be an integer from 1 to 5")
    if not isinstance(payload, str):
        raise ValueError("Missing or invalid 'payload' field")

    if redis_conn.zscore(PRIORITY_QUEUE_KEY, event_id) is not None:
        return

    type_rank = EVENT_TYPE_RANK[event_type]
    urgency_factor = 5 - urgency
    insert_index = redis_conn.incr(COUNTER_KEY)
    score = (urgency_factor * 1_000_000) + (type_rank * 1_000) + insert_index

    redis_conn.set(f"event:{event_id}", json.dumps(event))
    redis_conn.zadd(PRIORITY_QUEUE_KEY, {event_id: score})


def dequeue_event(redis_conn: redis.Redis) -> Optional[dict]:
    """Dequeue the highest priority event from Redis."""
    popped = redis_conn.zpopmin(PRIORITY_QUEUE_KEY, 1)
    if not popped:
        return []

    event_id, _ = popped[0]
    if isinstance(event_id, bytes):  # redis-py returns bytes
        event_id = event_id.decode()

    event_data_key = f"event:{event_id}"
    event_json = redis_conn.get(event_data_key)
    if not event_json:
        return None

    event_dict = json.loads(event_json)
    redis_conn.delete(event_data_key)
    return event_dict


In [ ]:
# tests


# Model Breaking Proof

#### Model Breaking Task URL: <https://rlhf-v3.turing.com/prompt/86807ed7-b233-4b9b-9379-78c0f2abd2bb?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F66754%2Fview>

#### Model code:

```python
import json
import redis
from typing import Optional, Dict

# Configure event type ranking. Lower rank => higher priority within same urgency.
EVENT_TYPE_RANK = {
    "alert": 0,
    "transaction": 1,
    "log": 2
    # any unknown type will default to rank=3 (lowest priority)
}

# Redis keys used by this queue
PRIORITY_QUEUE_KEY = "priority_event_queue"
COUNTER_KEY = "priority_event_counter"

# <Issue>: <Model uses `Dict` while function signature uses `dict`; mixing them breaks type consistency and may trigger linting errors>

def enqueue_event(redis_conn: redis.Redis, event: Dict) -> None:
    """
    Enqueue a new event into a Redis-Based Priority Queue.

    Priority is determined by:
      - Urgency: higher is better
      - Event Type: alert > transaction > log (unknown => lowest)
      - FIFO: tie-break on insertion order for same urgency & type
    """

    # Basic validation / safe defaults
    event_id = event.get("event_id")
    event_type = event.get("event_type", "")
    urgency = event.get("urgency", 0)

# <Issue>: <Silently skips bad input, no warning or error raised>

    # If any critical field is missing or urgency out of range, handle gracefully:
    if not event_id:
        # Could log or raise a custom exception; here we'll just skip enqueuing.
        return
    if urgency < 1 or urgency > 5:
        # Could log or raise a custom exception; here we'll just skip enqueuing.
        return

    # Check for duplicates; if the event already exists, skip.

# <Issue>: <Only checks Redis key, not ZSET membership (causes ghost events)>

    if redis_conn.exists(f"event:{event_id}"):
        return  # Already in queue

    # Determine event type rank (unknown => 3)
    type_rank = EVENT_TYPE_RANK.get(event_type, 3)

    # For urgency in [1..5], define a higher urgency => smaller "urgency_factor"
    # so that (urgency=5 => 0, urgency=1 => 4).
    urgency_factor = 5 - urgency

    # Get the next insertion counter, ensuring FIFO ordering for ties

    insert_index = redis_conn.incr(COUNTER_KEY)

    # Construct the sorted set score.
    # The exact numeric scaling is somewhat arbitrary; it just needs
    # to preserve the correct order. We reserve 1,000 for type rank,
    # another 1,000 for the insertion index, etc.
    # The overall structure:
    #   score = urgency_factor * 1_000_000 + type_rank * 1_000 + insert_index
    score = (urgency_factor * 1_000_000) + (type_rank * 1_000) + insert_index

    # Store the event data as JSON (string key "event:<event_id>")
    redis_conn.set(f"event:{event_id}", json.dumps(event))

    # Add the event_id to the priority queue (sorted set) with calculated score
    redis_conn.zadd(PRIORITY_QUEUE_KEY, {event_id: score})

# <Issue>: <Model uses `Dict` while function signature uses `dict`; mixing them breaks type consistency and may trigger linting errors>

def dequeue_event(redis_conn: redis.Redis) -> Optional[Dict]:
    """
    Dequeue a single event from the priority queue.
    Returns the event as a dictionary, or None if the queue is empty.
    """
    # Pop the event with the smallest score (highest priority) from the sorted set
    popped = redis_conn.zpopmin(PRIORITY_QUEUE_KEY, 1)
    if not popped:

# <Issue>: <Returns None instead of [] on empty dequeue (violates Edge case)>

        # Empty list => empty queue
        return None

    # popped is a list of tuples [(member, score)], we only popped 1
    event_id, _score = popped[0]

    # Retrieve the stored event JSON and remove it from Redis
    event_data_key = f"event:{event_id}"
    event_json = redis_conn.get(event_data_key)
    if event_json is None:
        # In a rare edge case if data got removed externally
        return None

    # Convert back to a Python dict
    event_dict = json.loads(event_json)

    # Delete the event data to clean up
    redis_conn.delete(event_data_key)

    return event_dict
```